In [1]:
import sys
sys.path.append('../')

%load_ext autoreload
%autoreload 2

import os
from custom_datasets.rollout_push_any import RolloutPushAnyMemDataset
from custom_datasets.pusht import PushTDataset
from concurrent.futures import ThreadPoolExecutor, wait

import re
from pathlib import Path
import zarr
import numpy as np
import torch
import torch.nn.functional as F
import imageio
from IPython.display import Video

from typing import Dict
import time

In [2]:
data_path = "/home/sm/Datasets/lerobot/pusht"

dataset = RolloutPushAnyMemDataset(data_path)

In [7]:
episode_index = 200
o, a, m = dataset[episode_index]
frames = (o.squeeze(1) * 255).type(torch.uint8).permute(0, 2, 3, 1).numpy()
frames = [frame for frame in frames]
Path("outputs/examples/1_load_lerobot_dataset").mkdir(parents=True, exist_ok=True)
video_path = f"outputs/examples/1_load_lerobot_dataset/episode_{episode_index}.mp4"
imageio.mimsave(video_path, frames, fps=10)

Video(video_path, embed=True, width=640, height=360)

## Profile

In [14]:
pusht_dataset = PushTDataset('/home/sm/Datasets/dynamo/datasets/dynamo_repro_datasets/pusht_dataset')

In [19]:
pusht_dataset.states[0][:10].size()

torch.Size([10, 5])

In [12]:
def profile_read(d, num_episodes=200):
    total_read_size = 0
    start = time.time()
    for episode_index in range(num_episodes):
        episode_index = episode_index % len(d)
        o = d[episode_index]
        total_read_size += len(o[0])
    
    total_time = time.time() - start
    print(f"total time: {total_time}, total_read_size: {total_read_size}")
    print(f"read speed: {total_time / total_read_size}")

In [21]:
profile_read(dataset, 1000)

total time: 160.41271138191223, total_read_size: 231994
read speed: 0.0006914519831629794


In [18]:
profile_read(pusht_dataset, 1000)

total time: 18.24829125404358, total_read_size: 124288
read speed: 0.00014682263174275537


In [51]:
dataset[0][0].size()

torch.Size([271, 1, 3, 224, 224])

In [52]:
new_frame_path

PosixPath('/home/sm/Datasets/lerobot/pusht/episode_frames')

In [53]:
def write_as_uint8(dataset):
    for episode_index in range(len(dataset)):
        o, a, m = dataset[episode_index]
        frames = (o.squeeze(1) * 255).type(torch.uint8)

        new_episode_frame_path = new_frame_path / f"episode_{episode_index}.pth"
        torch.save(frames, new_episode_frame_path)

In [54]:
write_as_uint8(dataset)